**Definitions**

- **Misc**
  - `i ∈ {1, ..., n}`: users  
  - `j ∈ {1, ..., m}`: products  
  - $k^{*}$: max number of KPIs met  

- **Parameters**
  - `p_ij ∈ [0, 1]`: propensity of user *i* to convert for product *j*  
  - `r_j`: RPC for product *j*  
  - `t_j`: KPI target revenue for product *j*  
  - `C`: user cap (max number of ads per user)  
  - `λ`: fatigue penalty weight  
  - `ε`: large constant  

- **Decision Variables**
  - `x_ij ∈ {0, 1}`: 1 if user *i* sees product *j*, 0 otherwise  
  - `y_j ∈ {0, 1}`: 1 if product *j* meets KPI target, 0 otherwise  

---

**Rationale**

- **Step 1** assumes *full revenue potential*, meaning all users who are shown a product generate revenue at that product’s full RPC. This is used to check if KPI targets can be met.
- **Step 2** uses *propensity-weighted revenue* (`p_ij * r_j`) to reflect actual expected revenue. This accounts for user differences in likelihood to convert and helps prioritize higher-propensity users.

### Step 1) Maximize number of KPIs met

$$
\max \sum_{j=1}^{m} y_j \text{, s.t.}
$$
$$
\sum_{j=1}^{m} x_{ij} \leq C \quad \forall i \in \{1, \dots, n\}
$$
$$
r_j \cdot \sum_{i=1}^{n} x_{ij} \geq t_j - (1 - y_j) \cdot \epsilon \quad \forall j \in \{1, \dots, m\}
$$
$$
x_{ij} \in \{0, 1\}, \quad y_j \in \{0, 1\}
$$

### Step 2) Maximize expected revenue (using propensity)
$$
\max \sum_{i=1}^{n} \sum_{j=1}^{m} p_{ij} \cdot r_j \cdot x_{ij}, \text{ s.t.}
$$
$$
\sum_{j=1}^{m} y_{j} \geq k^{*}
$$

In [1]:
!pip install pulp -q

In [2]:
import pulp
import numpy as np
import pandas as pd

In [3]:
def optimize_ad_assignment_only_exposure_pulp(propensity, rpc, kpi_targets, user_cap=1, epsilon=1e6):
    n_users, n_products = propensity.shape
    model = pulp.LpProblem("Ad_Strategy", pulp.LpMaximize)

    # decision variables
    # Tang: Pulp's decision variables 
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n_users) for j in range(n_products)), cat='Binary')
    y = pulp.LpVariable.dicts("y", (j for j in range(n_products)), cat='Binary')

    # if kpi is already met do not assign any users to this product
    # Tang: This means the algorithms pick off one that exceeds or reachs the kpis
    for j in range(n_products):
        if kpi_targets[j] <= 0:
            for i in range(n_users):
                model += x[i, j] == 0

    # if user-ad propensity is zero do not assign the ad to the user. 
    # Tang: Hard coded section. This may involves with overall obj function. 
    # Tang: Make sense, just don't assgin the ads that has 0 propensity in the first place.
    # Tang: This will make max_kpis_met lower but feasible. So, the calculated max_kpis_met will be more realistic. In other words, the condition is more realistic.
    for i in range(n_users):
        for j in range(n_products):
            if propensity[i,j] == 0:
                model += x[i, j] == 0

    # constraint: max number of ads per user
    # Do not send ads that exceed the cap per user.
    for i in range(n_users):
        model += pulp.lpSum(x[i, j] for j in range(n_products)) <= user_cap

    # constraint: number of ads * RPC >= target
    # Tang: The epsilon means we are trying to use big M method.
    for j in range(n_products):
        model += rpc[j] * pulp.lpSum(x[i, j] for i in range(n_users)) >= kpi_targets[j] - (1 - y[j]) * epsilon
        # model += pulp.lpSum(propensity[i, j] * rpc[j] * x[i, j] for i in range(n_users)) >= kpi_targets[j] - (1 - y[j]) * epsilon

    # 1) Adding constraint max number of KPIs met
    # Tang: This is the objective fuction for the first step, since we didn't define conditions here like >=, <= etc.
    # Tang: This can also be used to check number of kpis that can be reached in the single steps. So, it is useful to know wether the kpis are realistic.
    # Tang: model += pulp.lpSum(y[j] for j in range(n_products)) ##### Equation (1)
    model.setObjective(pulp.lpSum(y[j] for j in range(n_products))) ##### Alternative for Equation (1)

    # Tang: Solve for initial x_[i], y_[i]
    # Tang: PULP_CBC_CMD this is where a temporary file is located, and msg = 0 means supressing output.
    # Tang: CBC (Coin-or branch and cut)
    # Tang: Check number of possible kpis that can be met based on current constraints
    model.solve(pulp.PULP_CBC_CMD(msg=0))

    # Tang: Sum y[i] to get number of kpis based on (1). In other words, this gives us feasible kpis.
    max_kpis_met = sum(int(pulp.value(y[j])) for j in range(n_products))

    # constraint to lock in the number of KPIs met
    # Tang: add new conditions that the soltion must meet >= max_kpis_met based on the calculation
    # Tang: this is a new condition for kpis. 
    model += pulp.lpSum(y[j] for j in range(n_products)) >= max_kpis_met

    # 2) max expected revenue using propensity-weighted RPC
    # Tang: Set maximizing revenue objectives while also include num kpi constraints 
    model.setObjective(
        pulp.lpSum(
            propensity[i, j] * rpc[j] * x[i, j] 
            for i in range(n_users) for j in range(n_products)
        )
    )
    
    # Tang: CBC (Coin-or branch and cut)
    model.solve(pulp.PULP_CBC_CMD(msg=0))

    # result of the assignment
    assignment = np.zeros((n_users, n_products), dtype=int)
    for i in range(n_users):
        for j in range(n_products):
            assignment[i, j] = int(pulp.value(x[i, j]))

    assignment_df = pd.DataFrame(
        assignment,
        columns=[f"Product {chr(65 + j)}" for j in range(n_products)],
        index=[f"User {i+1}" for i in range(n_users)]
    )

    actual_exposure = assignment.sum(axis=0) * rpc
    kpi_report = pd.DataFrame({
        "Target KPI": kpi_targets,
        "Actual exposure (full RPC)": actual_exposure,
        "KPI met": actual_exposure >= kpi_targets
    }, index=[f"Product {chr(65 + j)}" for j in range(n_products)])

    total_expected_revenue = (assignment * propensity * rpc).sum()

    return assignment_df, kpi_report, total_expected_revenue, actual_exposure.sum()


### Test functions

In [4]:
propensity = np.array([
    [1, 0.2, 0],
    [0.7, 0.3, 0],
    [0.5, 0.5, 0.5],
    [0, 0, 0.3]
])
user_cap = 1
rpc = np.array([200, 300, 400])
kpi_targets = np.array([400, 300, 1200])
epsilon = 1e6

n_users, n_products = propensity.shape
model = pulp.LpProblem("Ad_Strategy", pulp.LpMaximize)

# decision variables
# Tang: Pulp's decision variables 
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n_users) for j in range(n_products)), cat='Binary')
y = pulp.LpVariable.dicts("y", (j for j in range(n_products)), cat='Binary')

# if kpi is already met do not assign any users to this product
# Tang: This means the algorithms pick off one that exceeds or reachs the kpis
for j in range(n_products):
    if kpi_targets[j] <= 0:
        for i in range(n_users):
            model += x[i, j] == 0

# if user-ad propensity is zero do not assign the ad to the user. 
# Tang: Hard coded section. This may involves with overall obj function. 
# Make sense, just don't assgin the ads that has 0 propensity in the first place.
for i in range(n_users):
    for j in range(n_products):
        if propensity[i,j] == 0:
            model += x[i, j] == 0

# constraint: max number of ads per user
# Do not send ads that exceed the cap per user.
for i in range(n_users):
    model += pulp.lpSum(x[i, j] for j in range(n_products)) <= user_cap

# constraint: number of ads * RPC >= target
# Tang: The epsilon means we are trying to use big M method.
for j in range(n_products):
    model += rpc[j] * pulp.lpSum(x[i, j] for i in range(n_users)) >= kpi_targets[j] - (1 - y[j]) * epsilon
    # model += pulp.lpSum(propensity[i, j] * rpc[j] * x[i, j] for i in range(n_users)) >= kpi_targets[j] - (1 - y[j]) * epsilon

# 1) max number of KPIs met
# Tang: PULP_CBC_CMD this is where a temporary file is located, and msg = 0 means supressing output.
# model += pulp.lpSum(y[j] for j in range(n_products))
model.setObjective(pulp.lpSum(y[j] for j in range(n_products)))
model.solve(pulp.PULP_CBC_CMD(msg=0))

1

In [5]:
print(y)
print(pulp.value(y[0]))

{0: y_0, 1: y_1, 2: y_2}
1.0


In [6]:
model

Ad_Strategy:
MAXIMIZE
1*y_0 + 1*y_1 + 1*y_2 + 0.0
SUBJECT TO
_C1: x_(0,_2) = 0

_C2: x_(1,_2) = 0

_C3: x_(3,_0) = 0

_C4: x_(3,_1) = 0

_C5: x_(0,_0) + x_(0,_1) + x_(0,_2) <= 1

_C6: x_(1,_0) + x_(1,_1) + x_(1,_2) <= 1

_C7: x_(2,_0) + x_(2,_1) + x_(2,_2) <= 1

_C8: x_(3,_0) + x_(3,_1) + x_(3,_2) <= 1

_C9: 200 x_(0,_0) + 200 x_(1,_0) + 200 x_(2,_0) + 200 x_(3,_0) - 1000000 y_0
 >= -999600

_C10: 300 x_(0,_1) + 300 x_(1,_1) + 300 x_(2,_1) + 300 x_(3,_1) - 1000000 y_1
 >= -999700

_C11: 400 x_(0,_2) + 400 x_(1,_2) + 400 x_(2,_2) + 400 x_(3,_2) - 1000000 y_2
 >= -998800

VARIABLES
0 <= x_(0,_0) <= 1 Integer
0 <= x_(0,_1) <= 1 Integer
0 <= x_(0,_2) <= 1 Integer
0 <= x_(1,_0) <= 1 Integer
0 <= x_(1,_1) <= 1 Integer
0 <= x_(1,_2) <= 1 Integer
0 <= x_(2,_0) <= 1 Integer
0 <= x_(2,_1) <= 1 Integer
0 <= x_(2,_2) <= 1 Integer
0 <= x_(3,_0) <= 1 Integer
0 <= x_(3,_1) <= 1 Integer
0 <= x_(3,_2) <= 1 Integer
0 <= y_0 <= 1 Integer
0 <= y_1 <= 1 Integer
0 <= y_2 <= 1 Integer

In [7]:
print(x)
print(
    pulp.value(x[0,0]), pulp.value(x[0,1]), pulp.value(x[0,2]), "\n", 
    pulp.value(x[1,0]), pulp.value(x[1,1]), pulp.value(x[1,2]), "\n", 
    pulp.value(x[2,0]), pulp.value(x[2,1]), pulp.value(x[2,2]), "\n", 
    pulp.value(x[3,0]), pulp.value(x[3,1]), pulp.value(x[3,2]), "\n", 
)

{(0, 0): x_(0,_0), (0, 1): x_(0,_1), (0, 2): x_(0,_2), (1, 0): x_(1,_0), (1, 1): x_(1,_1), (1, 2): x_(1,_2), (2, 0): x_(2,_0), (2, 1): x_(2,_1), (2, 2): x_(2,_2), (3, 0): x_(3,_0), (3, 1): x_(3,_1), (3, 2): x_(3,_2)}
1.0 0.0 0.0 
 0.0 1.0 0.0 
 1.0 0.0 0.0 
 0.0 0.0 1.0 



In [8]:
# Generate random propensity matrix with values between 0 and 1
propensity = np.random.rand(1000, 20)

# Generate random positive RPC and LIP arrays
rpc = np.random.uniform(100, 500, size=20)

# Generate random lip array with values between 700 and 1500
kpi_targets = np.random.uniform(700, 1500, size=20)

user_cap = 1

epsilon = 1e6

n_users, n_products = propensity.shape
model = pulp.LpProblem("Ad_Strategy", pulp.LpMaximize)

# decision variables
# Tang: Pulp's decision variables 
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n_users) for j in range(n_products)), cat='Binary')
y = pulp.LpVariable.dicts("y", (j for j in range(n_products)), cat='Binary')

# if kpi is already met do not assign any users to this product
# Tang: This means the algorithms pick off one that exceeds or reachs the kpis
for j in range(n_products):
    if kpi_targets[j] <= 0:
        for i in range(n_users):
            model += x[i, j] == 0

# if user-ad propensity is zero do not assign the ad to the user. 
# Tang: Hard coded section. This may involves with overall obj function. 
# Make sense, just don't assgin the ads that has 0 propensity in the first place.
for i in range(n_users):
    for j in range(n_products):
        if propensity[i,j] == 0:
            model += x[i, j] == 0

# constraint: max number of ads per user
# Do not send ads that exceed the cap per user.
for i in range(n_users):
    model += pulp.lpSum(x[i, j] for j in range(n_products)) <= user_cap

# constraint: number of ads * RPC >= target
# Tang: The epsilon means we are trying to use big M method.
for j in range(n_products):
    model += rpc[j] * pulp.lpSum(x[i, j] for i in range(n_users)) >= kpi_targets[j] - (1 - y[j]) * epsilon
    # model += pulp.lpSum(propensity[i, j] * rpc[j] * x[i, j] for i in range(n_users)) >= kpi_targets[j] - (1 - y[j]) * epsilon

# 1) max number of KPIs met
# Tang: PULP_CBC_CMD this is where a temporary file is located, and msg = 0 means supressing output.
# model += pulp.lpSum(y[j] for j in range(n_products))
model.setObjective(pulp.lpSum(y[j] for j in range(n_products)))
model.solve(pulp.PULP_CBC_CMD(msg=0))

1

Mock user feedback

Over multiple rounds (t = 1...T),

*   Update remaining_kpi = kpi_targets - cumulative_revenue_so_far
*   Track user exposure/ad fatigue
*   Adjust future fatigue


In [9]:
from utils.utils import multi_shot_simulation 

In [10]:
# inputs
propensity = np.array([
    [1, 0.2, 0],
    [0.7, 0.3, 0],
    [0.5, 0.5, 0],
    [0, 0.7, 0]
])
rpc = np.array([200, 300, 1200])
kpi_targets = np.array([400, 300, 1200])


## Run simulation
## Tang: No stop criteria
a = multi_shot_simulation(
    days=10,
    propensity=propensity,
    rpc=rpc,
    kpi_targets=kpi_targets,
    assign_fn=optimize_ad_assignment_only_exposure_pulp,
    user_cap = 1
)


----------- Day 0 -----------

Product-Level Info:
     RPC  KPI Target
P0   200         400
P1   300         300
P2  1200        1200

User Propensities:
         P0   P1   P2
User 1  1.0  0.2  0.0
User 2  0.7  0.3  0.0
User 3  0.5  0.5  0.0
User 4  0.0  0.7  0.0

----------- Day 1 -----------

Ad Assignment:
        P0  P1  P2
User 1   1   0   0
User 2   1   0   0
User 3   0   1   0
User 4   0   1   0

Ad Clicks:
        P0  P1  P2
User 1   1   0   0
User 2   0   0   0
User 3   0   0   0
User 4   0   0   0

decayed_propensity:
[[1.  0.2 0. ]
 [0.7 0.3 0. ]
 [0.5 0.5 0. ]
 [0.  0.7 0. ]]

fatigue matrix
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]

Live KPI Status:
     RPC  Target KPI  Current Rev  KPI Met
P0   200         400          200    False
P1   300         300            0    False
P2  1200        1200            0    False

----------- Day 2 -----------

Ad Assignment:
        P0  P1  P2
User 1   1   0   0
User 2   1   0   0
User 3   0   1   0
User 4   0   1   0

Ad Cl

In [11]:
# inputs
propensity = np.array([
    [1, 0.2, 0],
    [0.7, 0.3, 0],
    [0.5, 0.5, 0.5],
    [0, 0, 0.3]
])
rpc = np.array([200, 300, 400])
kpi_targets = np.array([400, 300, 1200])


## Run simulation
## Tang: No stop criteria
a = multi_shot_simulation(
    days=10,
    propensity=propensity,
    rpc=rpc,
    kpi_targets=kpi_targets,
    assign_fn=optimize_ad_assignment_only_exposure_pulp,
    user_cap = 1
)


----------- Day 0 -----------

Product-Level Info:
    RPC  KPI Target
P0  200         400
P1  300         300
P2  400        1200

User Propensities:
         P0   P1   P2
User 1  1.0  0.2  0.0
User 2  0.7  0.3  0.0
User 3  0.5  0.5  0.5
User 4  0.0  0.0  0.3

----------- Day 1 -----------

Ad Assignment:
        P0  P1  P2
User 1   1   0   0
User 2   1   0   0
User 3   0   1   0
User 4   0   0   1

Ad Clicks:
        P0  P1  P2
User 1   1   0   0
User 2   1   0   0
User 3   0   0   0
User 4   0   0   1

decayed_propensity:
[[1.  0.2 0. ]
 [0.7 0.3 0. ]
 [0.5 0.5 0.5]
 [0.  0.  0.3]]

fatigue matrix
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Live KPI Status:
    RPC  Target KPI  Current Rev  KPI Met
P0  200         400          400     True
P1  300         300            0    False
P2  400        1200          400    False

----------- Day 2 -----------

Ad Assignment:
        P0  P1  P2
User 1   0   1   0
User 2   0   1   0
User 3   0   0   1
User 4   0   0   1

Ad Clicks:
  

In [12]:
# inputs
propensity = np.array([
    [1, 0, 0],
    [0.7, 0.3, 0],
    [0.5, 0.5, 0.5],
    [0.8, 0.1, 0.2]
])
rpc = np.array([200, 300, 400])
kpi_targets = np.array([1000, 1000, 1000])

# run simulation
a = multi_shot_simulation(
    days=10,
    propensity=propensity,
    rpc=rpc,
    kpi_targets=kpi_targets,
    assign_fn=optimize_ad_assignment_only_exposure_pulp,
    user_cap = 1
)


----------- Day 0 -----------

Product-Level Info:
    RPC  KPI Target
P0  200        1000
P1  300        1000
P2  400        1000

User Propensities:
         P0   P1   P2
User 1  1.0  0.0  0.0
User 2  0.7  0.3  0.0
User 3  0.5  0.5  0.5
User 4  0.8  0.1  0.2

----------- Day 1 -----------

Ad Assignment:
        P0  P1  P2
User 1   1   0   0
User 2   1   0   0
User 3   0   0   1
User 4   1   0   0

Ad Clicks:
        P0  P1  P2
User 1   1   0   0
User 2   0   0   0
User 3   0   0   0
User 4   0   0   0

decayed_propensity:
[[1.  0.  0. ]
 [0.7 0.3 0. ]
 [0.5 0.5 0.5]
 [0.8 0.1 0.2]]

fatigue matrix
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

Live KPI Status:
    RPC  Target KPI  Current Rev  KPI Met
P0  200        1000          200    False
P1  300        1000            0    False
P2  400        1000            0    False

----------- Day 2 -----------

Ad Assignment:
        P0  P1  P2
User 1   1   0   0
User 2   1   0   0
User 3   1   0   0
User 4   1   0   0

Ad Clicks:
  